# Part 1: Spam or ham? (45 marks)

In the spam literature, an email that is **not** spam is called _ham_. 

Your task is to develop a Naive Bayes classifier to determine whether a given email is spam or ham.

## The data: Spam!

The following cell loads your training data set (called _training set_).

In [243]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",")
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]]


Your training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam (1) or ham (0). The remaining 54 columns are _features_ that you will use to build a classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value "1" if the keyword appears in the message and "0" otherwise.

## The model:  Naïve Bayes
Your [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier will distinguish between two classes:

* **$C = 1$ for spam messages **
* **$C = 0$ for ham messages. **


The classifier builds a model for the probability $P(C=c\ |\ message)$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate

\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \  P(C=c\ |\ message).
\end{equation}

Using Bayes' rule we can write

\begin{equation}
P(C=c\ |\ message) = \frac{P(message\ |\ C=c)P(C=c)}{P(message\ |\ C=1)P(C=1) + P(message\ |\ C=0)P(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it to get

\begin{equation}
P(C=c\ |\ message) \propto P(message\ |\ C=c)P(C=c).
\end{equation}


We represent a message using a binary [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as $\mathbf{w} = (w_1, ..., w_k)$, where $w_i = 1$ if the word $w_i$ appears in the message and $w_i = 0$ otherwise. We assume **class-conditional independence between occurences of known words** and can therefore write 

\begin{equation}
P(message\ |\ C=c) = \prod_{i = 1}^k P(w_i\ |\ C=c).
\end{equation}

The classifier now can be written as follows :

\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ [ P(C=c)   \prod_{i = 1}^k P(w_i\ |\ C=c) ].
\end{equation}


### Multinomial Naïve Bayes

Different naïve Bayes models differ in their distributional assumptions of $P(w\ |\ C=c)$, that is, the conditional likelihood of a word $w$ given a class $c$. We will model $P(w\ |\ C=c)$ using a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution). Intuitively, the multinomial distribution assumes that the words of a message are "drawn" independently from a bag of $k$ different keywords. Depending on the class membership $c$, each keyword has a probability $\theta_{class, word}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy, viagra... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment... $\}$, if the training data was mostly gathered from emails of researchers.

Both the class priors $P(C=c)$ and the class-conditional likelihoods $\theta_{c, w} = P(w\ |\ C=c)$ have to be estimated from the training data. The parameters $\theta_{c, w}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$, that is,

\begin{equation}
\theta_{c, w} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}

where $n_{c, w}$ is the number of times the word $w$ appears in messages of class $c$ in the training set, $n_{c}$ is the total count of words for all messages of class $c$, and $k$ is the number of features (key-words).

The likelihood of observing $\mathbf{w}$ in a message of class $c$ is proportional to
\begin{equation}
P(\mathbf{w}\ |\ C=c) \propto \prod_{i = 1}^k  (\theta_{c, i})^{w_i}.
\end{equation}

### Increasing numerical stability
We can increase the numerical stability of the algorithm by taking logarithms of the posterior distributions, that is,
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ log( P(C=c)   \prod_{i = 1}^k P(w_i\ |\ C=c) ) \\
 = \underset{c \in \{0,1\}}{argmax} \ [ log( P(C=c)) + \sum_{i = 1}^k w_i \ log(\theta_{c, i}) ].
\end{equation}

You will implement this more stable version of the algorithm in developing your classifier. 

## Part 1A: Estimate class priors (10 marks)

Define a function called `estimate_log_class_priors()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns a numpy array containing the **the logarithm** of the empirical class priors $P(C=c)$ for $c \in \{0, 1\}$.

In [244]:
def estimate_log_class_priors(data):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column, calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
    log(P(C=0)) and log(P(C=1))

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                 the first column contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """
        
    # Counts ones and zeros.
    ones = np.sum(data[:,0])
    zeros = len(data) - ones
    
    # Probabilities of ones and zeros.
    zeros_prob = zeros / len(data)
    ones_prob = ones / len(data)
    
    # Logs of probabilities.
    log_zero = np.log(zeros_prob)
    log_one = np.log(ones_prob)
    
    log_class_priors = np.array([log_zero, log_one])
    return log_class_priors

In [245]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors = estimate_log_class_priors(training_spam)
print("result", log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)


result [-0.48939034 -0.94933059]


## Part 1B: Estimate class-conditional likelihoods (10 marks)
Define a function called `estimate_log_class_conditional_likelihoods()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns **the logarithm** of the empirical class-conditional likelihoods $log(P(w_i | c_j))$ for all words $w_i$ and both classes ($j \in {0, 1}$). These parameters should be returned in a two-dimensional numpy-array with shape = `[num_classes, num_features]`.

Assume a multinomial feature distribution and use Laplace smoothing with $\alpha = 1$. 

Hint: many `numpy`-functions contain an `axis` argument. If you specify `axis=0`, you can perform column-wise (that is, feature-wise!) computations.

In [246]:
def estimate_log_class_conditional_likelihoods(data, alpha=1.0):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column and binary features, calculate the empirical
    class-conditional likelihoods, that is,
    log(P(w_i | c_j)) for all features i and both classes (j in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, n_features]

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    # Get messages
    messages = np.delete(data, 0, 1)
    
    # Number of features
    k = len(messages[0])
    
    # Initialise theta
    theta = np.zeros((2, k))
    
    # True classification array
    true_classes = data[:,0]
    
    # Splits data into spam and ham
    ones = data[:,0] == 1
    zeros = data[:,0] == 0
    s = data[ones,:] # Spam
    h = data[zeros,:] # Ham
    
    # Value of Nc
    total_zeros = np.sum(np.delete(h, 0, 1))
    total_ones = np.sum(np.delete(s, 0, 1))
    
    
    for i in range(k):
        class_1_count = 0
        class_0_count = 0
        feature_class = messages[:,i]
        
        for j in range(len(messages)):
            if feature_class[j] == 1:
                if true_classes[j] == 1: 
                    class_1_count += 1 # Count for class = 1 and feature = 1
                else:
                    class_0_count += 1 # Count for class = 0 and feature = 1
        
        prob_zero = (class_0_count + alpha) / (total_zeros + (k*alpha))
        prob_one = (class_1_count + alpha) / (total_ones + (k*alpha))
        
        theta[0][i] = np.log(prob_zero)
        theta[1][i] = np.log(prob_one)
    return theta

In [247]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)


## Part  1C: Classify e-mails (10 marks)

Having calculated the log class priors and the log class-conditional likelihoods for a given training set, define a function called `predict()`that takes a data set of new messages as input and predicts for each message whether it is spam or not. Note that the input should **not** contain a response variable.

Use your `predict()` function to classify the messages of your **training data set** `training_spam`. Compute the accuracy of your algorithm _in the training set_ by comparing your predictions to the true class values. Accuracy is simply defined as the proportion of true predictions made by your classifier. Store the accuracy of your naïve Bayes algorithm in a variable called `training_set_accuracy`.

In [477]:
def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    
    # Initialise class predictions numpy array
    class_predictions = np.zeros(len(new_data))

    for n in range(len(new_data)): # Iterate through each message
        message = new_data[n]
        pred = np.zeros(2)
        for j in range(len(log_class_priors)): # Iterate through both classes
            likelihoods = log_class_conditional_likelihoods[j]
            prior = log_class_priors[j]
            summed_probs = 0
            for i in range(len(message)): # Iterate through each feature in message
                summed_probs += message[i] * likelihoods[i]
            pred[j] = prior + summed_probs
        
        class_pred = np.argmax(pred) # Chooses index with highest value in pred
        class_predictions[n] = class_pred 
        
    return class_predictions

def accuracy(y_predictions, y_true):
    """
    Calculate the accuracy.
    
    :param y_predictions: a one-dimensional numpy array of predicted classes (0s and 1s).
    :param y_true: a one-dimensional numpy array of true classes (0s and 1s).
    
    :return acc: a float between 0 and 1 
    """
    count = 0
    for i in range(len(y_predictions)):
        if y_predictions[i] == y_true[i]:
            count += 1
    acc = count / len(y_predictions)
    return acc

In [478]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.
class_predictions = predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (1000,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))
       
# Check accuracy function
true_classes = training_spam[:, 0]
training_set_accuracy = accuracy(class_predictions, true_classes)
assert(isinstance(training_set_accuracy, float))
assert(0 <= training_set_accuracy <= 1)


## Part 1D: Classifying previously unseen data (10 marks).
The following cell loads a new set of 500 messages.

In [476]:
testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",")
print("Shape of the testing spam data set:", testing_spam.shape)
testing_spam

Shape of the testing spam data set: (500, 55)


array([[1., 0., 0., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

Use the naïve Bayes algorithm that you trained on `training_spam` in order to classify all messages in the `testing_spam` data set. Store the resulting accuracy in a variable called `testing_set_accuracy`.

In [475]:
priors = estimate_log_class_priors(training_spam)
likelihoods = estimate_log_class_conditional_likelihoods(training_spam)
data = np.delete(testing_spam, 0, 1)
predictions = predict(data, priors, likelihoods)
true_classes = testing_spam[:,0]
test_set_accuracy = accuracy(predictions, true_classes)

In [252]:
# This is a test cell. Do not delete or change.

## Part 1E: Discussion (5 marks).


In 3 sentences or less: Compare the accuracy of your classifier on the training set and on the test set. Are they different? If yes, how do you explain the difference?

The accuracy of the classifier on the training set is higher than what we find on the test set because the training data was used to train the classifier, thus the classifier had seen the data before. When using the model on the test set, the accuracy is lower because it is unseen data. 

# Part 2: Zero or One? (55 marks)

In this part of the coursework, you will develop a classifier for a different problem. All you will be given about this problem is a training data set. Your objective is to develop a classifier that will have the highest accuracy in unseen examples.

The following cell loads the training data set.

In [559]:
import numpy as np

training_data = np.loadtxt(open("data/training_data_part_2.csv"), delimiter=",")
print("Shape of the training data set:", training_data.shape)
print(training_data)

Shape of the training data set: (5000, 39)
[[0. 1. 1. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 1. 1. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 1. 0. ... 0. 0. 0.]]


The first column is again the response variable. The remaining 38 columns are binary features. You have multiple tasks:

(1) Your first task is to write a function called `train()` that takes `training_data` as input and returns all the fitted parameters of your model. Note that the fitted parameters of your model depend on the model you choose. For example, if you use a naïve Bayes classifier, you could return a list of class priors and conditional likelihoods. (This function will allow us to compute your model on the fly. We should be able to execute it in less than 10 minutes.) 

(2) Your second task is to provide a variable called `fitted_model` which stores the model parameters you found by executing your train() function on the training_data. If your train function takes more than 20 seconds to run, this variable should load precomputed parameter values (possibly from a file) rather than execute the train() function. 

In [570]:
import random as r
import matplotlib.pyplot as plt
%matplotlib inline


# A class to implement a K-nearest neighbour classifier
class KNN:
    
    
    def __init__(self, k=1):
        self.k = k # How many neighbours to consider
        self.train_true_classes = None
        
    def get_fitted_model(self, data):
        """Method avoids code duplication"""
        return data
        
    def classify(self, indexes):
        """Classifies the message given the indexes of the nearest neighbours"""
        classes = []
        for index in indexes:
            class_ = self.train_true_classes[index]
            classes.append(class_)
        return classes
               
    def get_indexes(self, n):
        """Gets the indexes of the nearest neighbours"""
        indexes = []
        for i in range(0, self.k):
            j = np.argmax(n)
            indexes.append(j)
            n = np.delete(n, j)
        return indexes
        
    def get_prediction(self, message, train_data):
        """Takes a message and the training data and finds the nearest neighbours"""
        neighbours = []
        for data in train_data:
            n = np.sum(message == data)
            neighbours.append(n)
        indexes = self.get_indexes(neighbours)
        class_list = self.classify(indexes)
        if np.sum(class_list) > (self.k / 2):
            pred = 1.0
        else:
            pred = 0
        return pred
        
    def class_predictions(self, test_data, train_data):
        """given the test data and train data it finds the class predictions for each message in test data"""
        self.train_true_classes = train_data[:,0] # Saves the true classes as a global variable
        class_predictions = np.zeros(len(test_data))
        for i in range(0, len(test_data)):
            message = test_data[i]
            prediction = self.get_prediction(message, train_data[:,1:])
            class_predictions[i] = prediction
        return class_predictions

In [571]:
# Naive Bayes class, using functions from the first question.

class NB:
    
    def __init__(self):
        pass
    
    def get_fitted_model(self, training_data):
        """Returns the priors and likelihood using the functions made in question 1"""
        priors = estimate_log_class_priors(training_data)
        likelihoods = estimate_log_class_conditional_likelihoods(training_data)
        fitted_model = (priors, likelihoods)
        return fitted_model
    
    def class_predictions(self, data, model):
        """Gets class predictions given the fitted model for naive bayes"""
        priors = model[0]
        likelihoods = model[1]
        class_predictions = predict(data, priors, likelihoods)
        return class_predictions

In [572]:
# Choose which classifier to use, default is k-nearest neighbour (KNN), 
# to see results of naive bayes uncomment it and comment out the KNN classifier

#classifier = NB() # Init Naive bayes classifier
classifier = KNN(k=1) # Init KNN classifier, with a value for k as argument, **should be an odd number to easily classify data**

In [579]:
def train(training_data):
    """
    Train a model on the training_data

    :param training_data: a two-dimensional numpy-array with shape = [200, 12] 
    
    :return fitted_model: any data structure that captures your model
    """
    
    fitted_model = classifier.get_fitted_model(training_data)
    return fitted_model

## Uncomment one of the following two lines depending on whether you want us to compute your model on the 
## fly or load a supplementary file.

fitted_model = train(training_data) 
# fitted_model = load(local_file)

(3) Your third task is to provide a function called `test()` that uses your `fitted_model` to classify the observations of `testing_data`. The `testing_data` is hidden and may contain any number of observations (rows). It contains 38 columns that have the same structure as the features of `training_data`. 

In [580]:
def test(testing_data, fitted_model):
    """
    Classify the rows of testing_data using a fitted_model. 

    :param testing_data: a two-dimensional numpy-array with shape = [n_test_samples, 11]
    :param fitted_model: the output of your train function.

    :return class_predictions: a numpy array containing the class predictions for each row
        of testing_data.
    """
    
    class_predictions = classifier.class_predictions(testing_data, fitted_model)
    return class_predictions

In [581]:
# Cross validation function to split up the training data into testing set and training set
def cross_validation(data, split=4):
    """Function for cross validation, splits the training data into train and test sets"""
    data_splits = np.split(data, split)
    i = r.randint(0, split - 1)
    test_data = data_splits[i]
    data_set = []
    for j in range(0, split):
        if j != i:
            data_set.append(data_splits[j])
    train_data = np.concatenate(data_set)
    return train_data, test_data

In [582]:
# Cell used to to test my classifier, 
# training data was split using a cross validation function (above)
# Choice of classifier is chosen a few cells above

data_train, data_test = cross_validation(training_data)
model = train(data_train)
class_predictions = test(data_test[:,1:], model)
true_classes = data_test[:,0]
test_accuracy = accuracy(class_predictions, true_classes)
print(test_accuracy)

0.9488


In [569]:
# This is a test cell. Do not delete or change. 
# You can use this cell to check whether the returned objects of your function are of the right data type.

# Test data types if input are the first 20 rows of the training_data.
class_predictions = test(training_data[:20, 1:], fitted_model)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
assert(class_predictions.shape == (20,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))


Describe in less than 10 sentences: Explain your classifier. Comment on its performance. What other alternative classifiers did you consider or experiment with? How does the performance of your classifier change as the size of the training set increases? You may want to include figures. 


I decided to implement a k-nearest neighbour (KNN) classifier, so I could classify test data by finding the most similar data within the training set. I tested the classifiers performance with varying sizes of the test set and k values of [1,3,5]. I made a cross validation function which provided me with random samples of the training data to use as the test and training set. This also means overfitting was avoided as using data that is present in the test set and training set would provide a 100% accuracy. 

I averaged the results out over 3 iterations so the results would be less noisey. I used an odd number to represent k, so there would be no 50/50 decisions needing to be made when classifying the data. The results show that restraining the region of the classifer (lower values of k) provided the highest accuracy. This is due to the fact that the feature set is reasonalbly large, thus some neighbours may not be very similar at all and could influence the overall classification, so by restricting it you take into consideration just the most similar data.
I was slightly surprised that at a test set size of 1000, the accuracy of the classifier would drop, to then peak at 1250 for all k.

<img src="images/graph2.png" style="width: 400px;"/>

I decided to compare the KNN, with the Naive bayes (NB) classifier made in question 1. NB uses a probalistic model to determine the outcome of the class whereas KNN does not, so it would be interesting to see how they compare. I used the best result of k=1 to compare against the performance of NB classifier. The NB classifier ranges from 0.87-0.9 and the KNN ranges from 0.93-0.96, showing that the KNN classifier performs better.

<img src="images/graph3.png" style="width: 400px;"/>

